Import Library

In [20]:
from PIL import Image
import os
import cv2
from tqdm import tqdm

1. Data Preprocessing

In [18]:
def zoom_out(image_path, zoom_factor):
    # Ambil nama gambar
    imgWithFormat = os.path.basename(image_path)
    imgName = imgWithFormat.split(".")[0]
    
    # Buka gambar 
    original_image = Image.open(image_path)
    original_width, original_height = original_image.size

    # Hitung ukuran baru setelah diperbesar
    new_width = int(original_width * zoom_factor)
    new_height = int(original_height * zoom_factor)
    
    # Perbesar gambar menggunakan metode resampling lanczos
    zoomed_out_image = original_image.resize((new_width, new_height), Image.LANCZOS)

    # Buat latar belakang hitam dengan ukuran sesuai gambar asli
    black_background = Image.new('RGB', (original_width, original_height), (0, 0, 0))

    # Hitung posisi gambar yang diperbesar di tengah latar belakang hitam
    x_offset = (original_width - new_width) // 2
    y_offset = (original_height - new_height) // 2
    
    # Tempelkan gambar yang diperbesar ke latar belakang hitam
    black_background.paste(zoomed_out_image, (x_offset, y_offset))
    
    return black_background
    
def convert_to_binary(image_path, dst_path, threshold=128):
    # Baca gambar menggunakan OpenCV
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Lakukan ambang biner
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    
    # Simpan gambar biner
    cv2.imwrite(dst_path, binary_image)


2. Zoom Out semua Gambar

In [13]:
def zoom_out_image_in_folder(path, dst_folder): 
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    
    for folder_name in os.listdir(path): # Loop nama folder
        folder_path = os.path.join(path, folder_name)
        output_folder_path = os.path.join(dst_folder, folder_name)
        
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)
        
        for file_name in os.listdir(folder_path): 
            file_path = os.path.join(folder_path, file_name)
            
            zoom_factors = [0.5, 0.6, 0.7, 0.8, 0.9, 1]
            
            img_name = file_name.split('.')[0]
            
            for factor in zoom_factors:
                # Lakukan zoom out
                zoomed_image = zoom_out(file_path, factor)
                
                output_file_path = os.path.join(output_folder_path, f"{img_name}_zoomed_out_{int(factor*100)}.jpg")
                
                # Simpan gambar
                zoomed_image.save(output_file_path)

for i in tqdm(range(100)):
    zoom_out_image_in_folder("dataset", "zoom_out_image")


100%|██████████| 100/100 [39:11<00:00, 23.51s/it]


3. Ubah Semua Gambar ke Bentuk Biner

In [26]:
def process_image_in_folder(path, dst_folder):
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
        
    # Masuk ke dataset
    for folder_name in os.listdir(path): 
        folder_path = os.path.join(path, folder_name)  
        output_folder_path = os.path.join(dst_folder, folder_name)
        
        if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)
        
        # Iterasi setiap file dalam folder
        for file_name in os.listdir(folder_path): 
            file_path = os.path.join(folder_path, file_name)
            
            # Periksa apakah file atau folder
            if os.path.isfile(file_path):
                # Tentukan path untuk menyimpan hasil biner
                output_file_path = os.path.join(output_folder_path, file_name)
                
                # Proses dan simpan gambar biner
                convert_to_binary(file_path, output_file_path, threshold=100)

for i in tqdm(range(100)):
    process_image_in_folder("zoom_out_image", "binary_image")


100%|██████████| 100/100 [33:22<00:00, 20.02s/it]


Ekstraksi Fitur Dengan HOG